### Prompt Template Classes
- Purpose: Provide a mechanism to construct prompts for models. Prompts come in varying shapes and sizes to suit different business needs. Prompt template classes address all these needs and help produce a Prompt Value class that's used as a prompt to models.
1. Remember from the PromptValue parameter that's input to all model classes
2. This session is all about generating that at runtime in a scalable and flexible way to address diverse business needs.
3. Product `PromptValue` from `PromptTemplate`
4. The classes that make up prompts module:
5. https://techblogs.cloudlex.com/langchain-iii-prompts-2df826c0ec3d#a487
6. For chat applications we use `ChatPromptTemplate` the concrete derived class from `BaseChatPromptTemplate`
7. If we wish to use f-strings in our prompt template then we use a concrete derived class of `StringPromptTemplate`

### Prompt Value Classes
- Purpose: Represent the value returned by a Prompt Template when its invoked.
1. Represents the actual prompt that sent to the model
2. Is generated at runtime when the prompt template is invoked
3. This is the class hierarchy: https://techblogs.cloudlex.com/langchain-iii-prompts-2df826c0ec3d#ee36

### Message Prompt Template Classes
1. Prompts employ messages prompt templates to create and manipulate prompt values at runtime
2. Here are the classes: https://techblogs.cloudlex.com/langchain-iii-prompts-2df826c0ec3d#ea75

### Message Classes
1. These represent the messages themselves.
2. They are produced by prompt message templates
3. Support for chunking large messages is in these set of classes.
4. Here are the classes: https://techblogs.cloudlex.com/langchain-iii-prompts-2df826c0ec3d#e750


### NOTE
1. The names of the classes in this module can be rather confusing.
2. A Prompt is made up of messages
3. A class whose name contains *MessagePromptTemplate* usually refers to template for the messages that make up the prompt
4. A class whose name contains *PromptTemplate* is the template of the overall prompt.
5. Check this diagram: https://techblogs.cloudlex.com/langchain-iii-prompts-2df826c0ec3d#a487

Prompt Template Classes -> Prompt Value
Message Prompt Template Classes -> Messages

In [4]:
import os
import configparser
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage

# set up config parser
config = configparser.ConfigParser()
config.read("../config.ini")  # holds secrets and keys


# load Groq config
groq = config["groq"]
os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')

# select the model.
model = ChatGroq(model="llama3-8b-8192")

print('setup done successfully')

# Basic Message Prompt Template
from langchain_core.prompts.chat import ChatMessagePromptTemplate

# This is the typical factory method
# Best is to check source code: https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/prompts/chat.py#L299
chatMessagePrompt = ChatMessagePromptTemplate.from_template(
    template= "Please give me flight options for {from_city} to {to_city}",
    role="travel agent"
)

# In the above role is required.

print(f"The type of Prompt Message Template is \n\t{type(chatMessagePrompt)}")

# Use the prompt message template to generate the message by providing values for all the replacement variables.
baseMessage = chatMessagePrompt.format(from_city="New Delhi", to_city="Mumbai", role="travel agent")

print(
    f"The type of message is: \n\t{type(baseMessage)}, \n\tand its __repr__ value is:  {baseMessage.__repr__()}"
)  

setup done successfully
The type of Prompt Message Template is 
	<class 'langchain_core.prompts.chat.ChatMessagePromptTemplate'>
The type of message is: 
	<class 'langchain_core.messages.chat.ChatMessage'>, 
	and its __repr__ value is:  ChatMessage(content='Please give me flight options for New Delhi to Mumbai', additional_kwargs={}, response_metadata={}, role='travel agent')


1. Lets try sending a system and human message to the model and get the response

In [5]:
# Create individual messages
sysMessage = SystemMessage("You are a chat bot who is an expert in political science")
humanMessage = HumanMessage(content="What is the capital of India?")

response = model.invoke([sysMessage, humanMessage])
print(response.content)

That's an easy one! The capital of India is New Delhi.


In [6]:
# Same example with a Message Prompt Template.

from langchain_core.prompts.chat import (SystemMessagePromptTemplate, HumanMessagePromptTemplate)

sysMessage = SystemMessagePromptTemplate.from_template(template='You are a chat bot who is an expert in political science')
humanMessage = HumanMessagePromptTemplate.from_template(template='What is the capital of {country}')


response = model.invoke([sysMessage.format(), humanMessage.format(country='USA')])

print(response.content)



That's an easy one! The capital of the United States of America is Washington, D.C. (short for District of Columbia).


## Chat Prompt Template - using a list of string.
This represents a conversation context of a chat. The conversation can contain many back and forth messages. This template threrefore has a colleciton of messages. We can supply the messages using tuples or creating message objects using Message Prompt Template.

We can provide any of the following type of messages:
1. system - Like instructions or a note or context. Anything that we as the developer want the model to use for understanding its role, its tasks and expectations and give sufficient context.
2. human - Encapsulates any responses or messages by the user
3. ai - Encapsulates any responses by the model. 
4. placeholder - A placeholder where the prompt will receive an optional list of messages at run time.
5. assistant - Gets created as an AI message internally

Apart from this there are message templates to represent messages to tools and their responses that go back to model. We shall cover them when we get to tools.

Below is an example of creating a chat prompt using string.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}")
    ]
)

print('Template Messages: ',template.messages.__repr__())
print('Template Object: ', template.__repr__())


promptValue = template.invoke(input={"name":"Arun", "user_input":"How is the weather today?"})
print('Prompt Value: ',promptValue.__repr__())

Template Messages:  [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful AI bot. Your name is {name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, how are you doing?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="I'm doing well, thanks!"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]
Template Object:  ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful AI bot. Your n

## ChatPrompt Template using objects

Lets try and create the same prompt using Message Prompt Templates instead.


In [8]:
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

# We use message prompt template as we have a replacement variable.
systemMessage = SystemMessagePromptTemplate.from_template("You are a helpful AI bot. Your name is {name}.")

# No need to use message prompt template as no replacement variable.
humanMessage1 = HumanMessage("Hello, how are you doing?")

aimessage = AIMessage("I'm doing well, thanks!")

humanMessage2 = HumanMessagePromptTemplate.from_template("{user_input}")

template = ChatPromptTemplate.from_messages([
    systemMessage, humanMessage1, aimessage, humanMessage2
])

promptValue = template.invoke(input={"name":"Arun", "user_input":"How is the weather today?"})

print('Prompt Value: ', promptValue.__repr__())

model.invoke(promptValue)

Prompt Value:  ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Arun.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='How is the weather today?', additional_kwargs={}, response_metadata={})])


AIMessage(content="I'm an artificial intelligence, I don't have the capability to perceive the physical world, so I don't have access to real-time weather information. However, I can try to help you find out the weather in your area! Can you please tell me your location or the city you're interested in knowing the weather for?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 58, 'total_tokens': 124, 'completion_time': 0.055, 'prompt_time': 0.007043958, 'queue_time': 0.007122091, 'total_time': 0.062043958}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-c2fc1c36-95f0-4da4-b522-1ff0ced2bf6d-0', usage_metadata={'input_tokens': 58, 'output_tokens': 66, 'total_tokens': 124})

## Chat Prompt Template with image
Lets say you want to send an iamge to a Model along with a question associated with the image

In [10]:
from langchain_core.prompts.image import ImagePromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage


# Lets create the Image Prompt Template
imagePrompt = ImagePromptTemplate(input_variables=["imageName"], template={"url":f'https://www.hdcarwallpapers.com/walls/ford_gt_2016-HD.jpg', "detail":"low"})

# Just to check we are doing this right - lets invoke and see if the variables are replaced
promptValue = imagePrompt.invoke(input={"imageName":"good_pic.jpeg"})
print('Image Prompt',imagePrompt.__repr__())
print('Prompt Value', promptValue.__repr__())
print("ImageURL object", promptValue.image_url)


# Now lets wrap our Image Prompt Template within our Human Message Prompt Template
humanMessage1 = HumanMessagePromptTemplate(prompt=[imagePrompt])
print('Human Message1', humanMessage1.__repr__())

# And since there is no replacement variable we create the message directly.
humanMessage2 = HumanMessage("Describe this image in brief")
print('Human Message2', humanMessage2.__repr__())

# Now we can use both these in our Chat Prompt Template
chatPrompt = ChatPromptTemplate.from_messages([
    humanMessage1, humanMessage2
])
print('Chat prompt: ', chatPrompt.__repr__())

# And invoke will substitute the values and prepare the prompt to be sent to model
promptValue = chatPrompt.invoke(input={"imageName":"my_pic.png"})
print('Prompt Value: ',promptValue.__repr__())

model.invoke(promptValue)# Try when you have an image.





Image Prompt ImagePromptTemplate(input_variables=['imageName'], input_types={}, partial_variables={}, template={'url': 'https://www.hdcarwallpapers.com/walls/ford_gt_2016-HD.jpg', 'detail': 'low'})
Prompt Value ImagePromptValue(image_url={'detail': 'low', 'url': 'https://www.hdcarwallpapers.com/walls/ford_gt_2016-HD.jpg'})
ImageURL object {'detail': 'low', 'url': 'https://www.hdcarwallpapers.com/walls/ford_gt_2016-HD.jpg'}
Human Message1 HumanMessagePromptTemplate(prompt=[ImagePromptTemplate(input_variables=['imageName'], input_types={}, partial_variables={}, template={'url': 'https://www.hdcarwallpapers.com/walls/ford_gt_2016-HD.jpg', 'detail': 'low'})], additional_kwargs={})
Human Message2 HumanMessage(content='Describe this image in brief', additional_kwargs={}, response_metadata={})
Chat prompt:  ChatPromptTemplate(input_variables=['imageName'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=[ImagePromptTemplate(input_variables=['imageName'], inpu

BadRequestError: Error code: 400 - {'error': {'message': 'message[0].content must be a string', 'type': 'invalid_request_error'}}

### Prompts with examples
1. At times we want to provide examples with our prompts.
2. Langchain has extensive support for examples.
3. Check a useful diagram here: https://techblogs.cloudlex.com/langchain-iii-prompts-2df826c0ec3d#ea75#1dc7
4. You can provide examples directly with your code or usually you want to pick examples from a store of examples.

In [ ]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
examples = [
    {
        "type":"animals",
        "que":"Do lions hunt in a pack?",
        "ans":"Yes lions, unlike tigers hunt as a group"
    },
    {
        "type":"corporates",
        "que":"Who is the founder of Tesla Motors?",
        "ans": "Elon Musk"
    },
    {
        "type": "geography",
        "que":"What are the mountain ranges in west India?",
        "ans": "They are western ghats"
    },
    {
        "type":"synonyms",
        "que":"stoic",
        "ans":"quiet"
    }
]

examples = [
    {
        "que":"Do lions hunt in a pack?",
        "ans":"Yes lions, unlike tigers hunt as a group"
    },
    {
        "que":"Who is the founder of Tesla Motors?",
        "ans": "Elon Musk"
    },
    {
        "que":"What are the mountain ranges in west India?",
        "ans": "They are western ghats"
    },
    {
        "que":"stoic",
        "ans":"quiet"
    }
]


prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt = PromptTemplate.from_template("Input: {que} -> Output: {ans}"),
    prefix = "Please answer the following question based on the examples provided.",
    suffix = "Input: {user_question} -> Output: ",
    input_variables=["user_question"]
)

promptValue = prompt.format(user_question="What is the capital of Japan")

print(promptValue.__repr__())

model.invoke(promptValue)


'Please answer the following question based on the examples provided.\n\nInput: Do lions hunt in a pack? -> Output: Yes lions, unlike tigers hunt as a group\n\nInput: Who is the founder of Tesla Motors? -> Output: Elon Musk\n\nInput: What are the mountain ranges in west India? -> Output: They are western ghats\n\nInput: stoic -> Output: quiet\n\nInput: What is the capital of Japan -> Output: '


AIMessage(content='Based on the examples provided, the output would be:\n\nTōkyō', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 101, 'total_tokens': 117, 'completion_time': 0.013333333, 'prompt_time': 0.004531913, 'queue_time': 0.009829776, 'total_time': 0.017865246}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e19a4f6-f0ec-46e1-9583-7d16a7102964-0', usage_metadata={'input_tokens': 101, 'output_tokens': 16, 'total_tokens': 117})

In [ ]:
from typing import Any, Dict, List
from langchain_core.example_selectors import LengthBasedExampleSelector, BaseExampleSelector
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
examples = [
    {
        "type":"animals",
        "que":"Do lions hunt in a pack?",
        "ans":"Yes lions, unlike tigers hunt as a group"
    },
    {
        "type":"corporates",
        "que":"Who is the founder of Tesla Motors?",
        "ans": "Elon Musk"
    },
    {
        "type": "geography",
        "que":"What are the mountain ranges in west India?",
        "ans": "They are western ghats"
    },
    {
        "type":"synonyms",
        "que":"stoic",
        "ans":"quiet"
    }
    
]
class MyExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    def add_example(self, example: Dict[str, str]) -> Any:
        return super().add_example(example)

    def select_examples(self, input_variables) -> List[dict]:
        selected_examples = []
        typeOfQue = input_variables["typeOfQue"]
        print(input_variables)
        for example in self.examples:
            if example["type"] == typeOfQue:
                selected_examples.append(example)

        return selected_examples

        
# selector = LengthBasedExampleSelector(examples=examples, example_prompt= PromptTemplate.from_template("Input: {que} -> Output: {ans}"), max_length=8)
selector = MyExampleSelector(examples=examples)


prompt = FewShotPromptTemplate(
    example_selector=selector, # now instead of using examples directly we use selector
    example_prompt = PromptTemplate.from_template("Input: {que} -> Output: {ans}"),
    prefix = "Please answer the following question based on the examples provided.",
    suffix = "Input: {user_question} -> Output: ",
    input_variables=["user_question", "typeOfQue"]
)



promptValue = prompt.format(user_question="What is the capital of Japan", typeOfQue="geography")
print(promptValue.__repr__())


{'user_question': 'What is the capital of Japan', 'typeOfQue': 'geography'}
'Please answer the following question based on the examples provided.\n\nInput: What are the mountain ranges in west India? -> Output: They are western ghats\n\nInput: What is the capital of Japan -> Output: '


In [ ]:
examples = [
    {
        "type":"animals",
        "que":"Do lions hunt in a pack?",
        "ans":"Yes lions, unlike tigers hunt as a group"
    },
    {
        "type":"corporates",
        "que":"Who is the founder of Tesla Motors?",
        "ans": "Elon Musk"
    },
    {
        "type": "geography",
        "que":"What are the mountain ranges in west India?",
        "ans": "They are western ghats"
    },
    {
        "type":"synonyms",
        "que":"stoic",
        "ans":"quiet"
    }
    
]
l = []
for example in examples:
    if example["type"] == "geography":
        l.append(example)

print(l)

"HHH".lower()

[{'type': 'geography', 'que': 'What are the mountain ranges in west India?', 'ans': 'They are western ghats'}]


'hhh'